In [2]:
import pickle
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import numpy as np

np.random.seed(42)

In [3]:
# path = '/Users/thomas/Downloads/nturgb+d_skeletons'
path = 'D:\\Datasets\\Motion Privacy\\NTU RGB+D 120\\Skeleton Data'

In [4]:
# Attempt to load X and Y from pickle before generating them
X = {}
try:
    print('Attempting to load X from pickle')
    with open('X.pkl', 'rb') as f:
        X = pickle.load(f)
    print('X loaded from pickle')
except:
    print('Could not load X and Y, generating them now')
    
    # Read the files
    files = [f for f in listdir(path) if isfile(join(path, f))]

    # Get stats for each file based on name
    files_ = []
    for file in files:
        data = {'file': file,
                's': file[0:4],
                'c': file[4:8],
                'p': file[8:12],
                'r': file[12:16],
                'a': file[16:20]
                }
        files_.append(data)

    # Generate X and Y
    for file_ in tqdm(files_, desc='Files Parsed', position=0):
        try:
            file = join(path, file_['file'])
            data = open(file, 'r')
            lines = data.readlines()
            frames_count = int(lines.pop(0).replace('\n', ''))
            file_['frames'] = frames_count
        except UnicodeDecodeError: # .DS_Store file
            print('UnicodeDecodeError: ', file)
            continue

        # Add filename as key to X
        X[file_['file']] = []

        # Skip file if 2 actors
        if lines[0].replace('\n', '') != '1': continue

        for f in tqdm(range(frames_count), desc='Frames Parsed', position=1, leave=False):
            try:
                # Get actor count
                actors = int(lines.pop(0).replace('\n', ''))
            
                # Get actor info
                t = lines.pop(0)

                # Get joint count
                joint_count = int(lines.pop(0).replace('\n', ''))

                # Get joint info
                d = []
                for j in range(joint_count):
                    joint = lines.pop(0).replace('\n', '').split(' ')
                    d.extend(joint[0:3])

                # Skip if not 25 joints
                if len(d) != 75: continue

                # Convert to numpy array
                d = np.array(d)

                # Append to X and Y
                X[file_['file']].append(d)
            except:
                break
        
        # Convert to numpy array
        X[file_['file']] = np.array(X[file_['file']], dtype=np.float16)

    print('X Generated, saving to pickle...')

    # Save the data
    with open('X.pkl', 'wb') as f:
        pickle.dump(X, f)

    print('X Saved to pickle')


Attempting to load X from pickle
Could not load X and Y, generating them now


Files Parsed: 100%|██████████| 114480/114480 [1:05:17<00:00, 29.22it/s]


X Generated, saving to pickle...
X Saved to pickle
